# for mac we need to enable MPS

In [1]:
!pip3 install torch torchvision torchaudio


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# check if mps is available

In [2]:
import torch

if torch.backends.mps.is_available():
    print("MPS backend is available!")
else:
    print("MPS backend is not available. Ensure macOS 12.3+ and PyTorch 1.12.0+.")


MPS backend is available!


# set the device to MPS

In [3]:

# Check if MPS is available and set it as the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


Using device: mps


# Installing libraries

In [4]:
!pip3 install transformers -U
!pip3 install accelerate
!pip3 install evaluate -U
!pip3 install datasets -U
!pip3 install wandb
!pip3 install scikit-learn
# import wandb


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Wandb setup

In [ ]:
# wandb.login(key='<api key')

In [ ]:
# wandb.init(project='project', name='<name>')

# google drive setup

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls "/content/drive/MyDrive/Project"

Mounted at /content/drive
dataset			 liar_dataset		 project_v1_1_new.ipynb  project_v1.5.ipynb
fakenews		 liar_distilbert_output  project_v1_2.ipynb
liar2_dataset		 project.ipynb		 project_v1_3.ipynb
liar2_distilbert_output  project_v1.1.ipynb	 project_v1_4.ipynb


# loading the train dataset

In [5]:
from datasets import load_dataset

# Specify the saved path in your Google Drive
save_path = './fake-news/train.csv'

dataset = load_dataset("csv", data_files=save_path)

# dataset inspections
print(dataset)


/Users/ari/PycharmProjects/BertProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ari/PycharmProjects/BertProject/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'author', 'text', 'label'],
        num_rows: 20800
    })
})


# Importing libraries

In [23]:
# from datasets import Dataset, ClassLabel
# from datasets import load_dataset, DatasetDict
import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification,
from transformers import RobertaTokenizerFast, RobertaModel, Trainer, TrainingArguments
# from itertools import chain
import torch.nn as nn
import pandas as pd
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# import evaluate # hugging face evaluation metrics
# from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import Value
# from datasets import Features, Sequence





# split the train set

In [7]:
# Split the dataset into training and validation sets
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

# assign the splits
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

# Inspect the splits
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Training set size: 16640
Validation set size: 4160


# rename column 'label'

In [8]:
train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")

# check the dataset balance

In [9]:
labels = [example['labels'] for example in train_dataset]
fake_count = labels.count(1)
real_count = labels.count(0)
total = len(labels)
print(f"Train Dataset:")
print(f"Total Samples: {total}")
print(f"Fake News (1): {fake_count} ({(fake_count/total)*100:.2f}%)")
print(f"Real News (0): {real_count} ({(real_count/total)*100:.2f}%)\n")

labels = [example['labels'] for example in val_dataset]
fake_count = labels.count(1)
real_count = labels.count(0)
total = len(labels)
print(f"Train Dataset:")
print(f"Total Samples: {total}")
print(f"Fake News (1): {fake_count} ({(fake_count/total)*100:.2f}%)")
print(f"Real News (0): {real_count} ({(real_count/total)*100:.2f}%)\n")

Train Dataset:
Total Samples: 16640
Fake News (1): 8308 (49.93%)
Real News (0): 8332 (50.07%)

Train Dataset:
Total Samples: 4160
Fake News (1): 2105 (50.60%)
Real News (0): 2055 (49.40%)



# tokenizer

In [11]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# cast label values to flaot

In [12]:
new_features = train_dataset.features.copy()
new_features['labels'] = Value('float32')
train_dataset = train_dataset.cast(new_features)

new_features = val_dataset.features.copy()
new_features['labels'] = Value('float32')
val_dataset = val_dataset.cast(new_features)

# Tokenizer function

In [13]:
def tokenize_function(example):
    tokenized = tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)
    return tokenized

# removing empty rows

In [14]:
print(train_dataset.features)
# Filter rows where 'text' is None or empty
train_dataset = train_dataset.filter(lambda x: x['text'] is not None and x['text'].strip() != "")
val_dataset = val_dataset.filter(lambda x: x['text'] is not None and x['text'].strip() != "")


{'id': Value(dtype='int64', id=None), 'title': Value(dtype='string', id=None), 'author': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None), 'labels': Value(dtype='float32', id=None)}


# tokenizing and formatting the datasets

In [15]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

val_dataset = val_dataset.map(tokenize_function, batched=True)
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map: 100%|██████████| 4138/4138 [00:10<00:00, 393.44 examples/s]


# fakestack model

In [25]:

class FakeStackModel(nn.Module):
    def __init__(self, freeze_bert=False):
        super(FakeStackModel, self).__init__()
        self.bert = RobertaModel.from_pretrained('roberta-base')
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        # Initial drop out
        self.dropout = nn.Dropout(0.3)

        # 1x1 Convolution for Skip Connection
        self.residual_conv = nn.Conv1d(in_channels=768, out_channels=64, kernel_size=1)

        # CNN Layers
        self.conv1 = nn.Conv1d(in_channels=768, out_channels=64, kernel_size=5, padding=2)
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=5, padding=2)
        self.residual_conv2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=1)
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, padding=1)
        self.residual_conv3 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=1)
        self.maxpool3 = nn.MaxPool1d(kernel_size=2)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = nn.Conv1d(in_channels=16, out_channels=8, kernel_size=3, padding=1)
        self.residual_conv4 = nn.Conv1d(in_channels=16, out_channels=8, kernel_size=1)
        self.maxpool4 = nn.MaxPool1d(kernel_size=2)
        self.dropout4 = nn.Dropout(0.2)

        # LSTM layer
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        # Linear layer
        self.fc = nn.Linear(128, 1)  # binary


    def forward(self, input_ids, attention_mask, labels=None):
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = bert_outputs.last_hidden_state  # Shape: [batch_size, seq_len, 768]

        x = self.dropout(x)
        # permute to prepare for CNN
        x = x.permute(0, 2, 1)  # Shape: [batch_size, 768, seq_len]

        residual = self.residual_conv(x)

        x= self.conv1(x)
        x = nn.functional.relu(x)
        residual = nn.functional.adaptive_avg_pool1d(residual, x.shape[-1])
        x = x + residual


        x = self.maxpool1(x)
        x = self.dropout1(x)

        residual = self.residual_conv2(residual)

        x = self.conv2(x)
        x = nn.functional.relu(x)
        residual = nn.functional.adaptive_avg_pool1d(residual, x.shape[-1])
        x = x + residual

        x = self.maxpool2(x)
        x = self.dropout2(x)

        residual = self.residual_conv3(residual)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        residual = nn.functional.adaptive_avg_pool1d(residual, x.shape[-1])
        x = x + residual

        x = self.maxpool3(x)
        x = self.dropout3(x)

        residual = nn.functional.adaptive_avg_pool1d(residual, x.shape[-1])
        residual = self.residual_conv4(residual)
        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.maxpool4(x)
        x = self.dropout4(x)

        x = x.permute(0, 2, 1)  # Shape: [batch_size, seq_len, 256]
        x, (hh, cn) = self.lstm(x)  # Shape: [batch_size, seq_len, 128]
        x = hh[-1]  # Take the last hidden state


        logits = self.fc(x)  # Shape: [batch_size, 1]
        logits = logits.squeeze(-1)  # Remove the extra dimension

        loss = None
        # Compute loss if labels are provided
        if labels is not None:
            labels = labels.float()  # Convert labels to float
            loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.5], device=logits.device))  # Adjust pos_weight if needed
            loss = loss_fn(logits.view(-1), labels.float().to(logits.device))  # Flatten for BCEWithLogitsLoss


        return {'loss':loss, 'logits': logits}


# training arguments

In [18]:
# training arguments hyperprameters
from torch.optim import AdamW

training_args = TrainingArguments(
    output_dir="./liar_roberta_output",  # Where to save the model
    evaluation_strategy="epoch",         # Evaluate at the end of every epoch
    save_strategy="epoch",               # Save model at the end of every epoch
    learning_rate=2e-5,                  # Learning rate
    per_device_train_batch_size=8,      # Batch size for training
    per_device_eval_batch_size=16,       # Batch size for evaluation
    num_train_epochs=6,                  # Number of epochs
    weight_decay=0.01,                   # Regularization
    logging_dir="./logs",                # Logging directory
    logging_steps=50,                    # Log every 50 steps
    save_total_limit=1,                  # Keep only the last 2 checkpoints
    load_best_model_at_end=True,         # Load the best model when done
    metric_for_best_model="accuracy",    # Metric to determine the best model
    greater_is_better=True,              # Higher metric values are better
    report_to='none',
    # report_to='wandb',  #  W&B logging
    # run_name='<name of wandb instance>',  # W&B run name
    gradient_accumulation_steps=1,  # Gradient accumulation steps
    fp16=False,
    lr_scheduler_type='linear',
    optim='adamw_torch'
)



/Users/ari/PycharmProjects/BertProject/.venv/lib/python3.9/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# compute metric function

In [19]:
import numpy as np
# dynamically selecting the best threshold, as we do NOT have access to test dataset labels
best_threshold1 = 0.41
def compute_metrics(eval_pred):
    global best_threshold1
    logits, labels = eval_pred
    # # Convert logits to probabilities
    if isinstance(logits, tuple):
        logits = logits[0]

    # Apply sigmoid to logits to get probabilities
    probs = torch.sigmoid(torch.tensor(logits)).numpy()

    # Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
    thresholds = np.linspace(0.1, 0.9, num=9)
    best_threshold = 0.41
    best_f1 = 0

    for threshold in thresholds:
        preds = (probs > threshold).astype(int)
        f1 = f1_score(labels, preds, average='binary', zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    # Use the best threshold
    preds = (probs > best_threshold).astype(int)
    best_threshold1 = best_threshold
    # Ensure labels are integers
    labels = labels.astype(int)

    # Calculate metrics with zero_division=0 to handle cases with no positive predictions
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='binary', zero_division=0)
    recall = recall_score(labels, preds, average='binary', zero_division=0)
    f1 = f1_score(labels, preds, average='binary', zero_division=0)

    # Optional: printing the unique predictions and labels for debugging
    # unique_preds = set(preds)
    # unique_labels = set(labels)
    # print(f"Unique Predictions: {unique_preds}")
    # print(f"Unique Labels: {unique_labels}")
    # print("Sample Predictions:", preds[:50])
    # print("Sample Labels:", labels[:50])

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


# data collector

In [20]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# Trainer setup

In [26]:
from transformers import EarlyStoppingCallback
from torch.utils.data import DataLoader

# anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)
# early stopping function
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,
    early_stopping_threshold=0.0
)
model = FakeStackModel().to(device)  # moving to MPS
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# training the model

In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.027900,0.032463,0.996617,0.997117,0.996159,0.996638
2,0.019700,0.048936,0.993233,0.998061,0.988478,0.993247
3,0.002800,0.016716,0.998067,0.998558,0.997600,0.998079
4,0.001800,0.018007,0.998067,0.998558,0.997600,0.998079
5,0.025900,0.013173,0.998308,0.998559,0.998080,0.998319
6,0.001100,0.013359,0.998550,0.998560,0.998560,0.998560


Unique Predictions: {np.int64(0), np.int64(1)}
Unique Labels: {np.int64(0), np.int64(1)}
Sample Predictions: [1 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1
 0 0 0 1 0 0 0 1 1 0 0 0 1]
Sample Labels: [1 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1
 0 0 0 1 0 0 0 1 1 0 0 0 1]
Unique Predictions: {np.int64(0), np.int64(1)}
Unique Labels: {np.int64(0), np.int64(1)}
Sample Predictions: [1 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1
 0 0 0 1 0 0 0 1 1 0 0 0 1]
Sample Labels: [1 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1
 0 0 0 1 0 0 0 1 1 0 0 0 1]
Unique Predictions: {np.int64(0), np.int64(1)}
Unique Labels: {np.int64(0), np.int64(1)}
Sample Predictions: [1 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1
 0 0 0 1 0 0 0 1 1 0 0 0 1]
Sample Labels: [1 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1
 0 0 0 1 0 0 0 1 1 0 0 0 1]
Unique Pre

TrainOutput(global_step=12414, training_loss=0.04445136705914907, metrics={'train_runtime': 7563.8495, 'train_samples_per_second': 13.125, 'train_steps_per_second': 1.641, 'total_flos': 0.0, 'train_loss': 0.04445136705914907, 'epoch': 6.0})

# model evaluation

In [28]:
eval_results = trainer.evaluate()
print(eval_results)

Unique Predictions: {np.int64(0), np.int64(1)}
Unique Labels: {np.int64(0), np.int64(1)}
Sample Predictions: [1 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1
 0 0 0 1 0 0 0 1 1 0 0 0 1]
Sample Labels: [1 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1
 0 0 0 1 0 0 0 1 1 0 0 0 1]
{'eval_loss': 0.013359030708670616, 'eval_accuracy': 0.9985500241662639, 'eval_precision': 0.9985597695631301, 'eval_recall': 0.9985597695631301, 'eval_f1': 0.9985597695631301, 'eval_runtime': 31.2992, 'eval_samples_per_second': 132.208, 'eval_steps_per_second': 8.275, 'epoch': 6.0}


# saving the model (change to absolute path if using cloud)

In [29]:
# Save the final model manually (just to ensure it's explicitly saved)
trainer.save_model("./saved_Roberta_model")

# Load and process the test dataset for submission

## tokenization of the valid entries in test dataset

In [31]:
test_dataset = load_dataset("csv", data_files='./fake-news/test.csv')['train']

# Identify entries with valid text
def has_valid_text(example):
    return example['text'] is not None and example['text'].strip() != ""

# Create boolean mask for entries with valid text
valid_text_mask = [has_valid_text(example) for example in test_dataset]

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)
# Add a dummy 'labels' column with zeros to the test dataset (there is no labels column in test dataset)
test_dataset = test_dataset.map(lambda x: {'labels': 0})
# Tokenizing the valid entries
test_dataset_valid = test_dataset.filter(lambda example: has_valid_text(example))
test_dataset_valid = test_dataset_valid.map(tokenize_function, batched=True)

# Set the dataset format for PyTorch
test_dataset_valid.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Make predictions on entries with valid text
test_results = trainer.predict(test_dataset_valid)
test_logits = test_results.predictions

# Convert logits to probabilities and apply threshold (threshold was calculated during validation phase)
test_probs = torch.sigmoid(torch.tensor(test_logits)).numpy()
test_predictions_valid = (test_probs > best_threshold1).astype(int)


Map: 100%|██████████| 5162/5162 [00:14<00:00, 368.46 examples/s]


Unique Predictions: {np.int64(0), np.int64(1)}
Unique Labels: {np.int64(0)}
Sample Predictions: [0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 1 1 1 1 0 1 0 0 1 0 1 0 0 1 0 0 1
 1 0 0 0 1 0 0 1 0 0 1 1 0]
Sample Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


# Handling empty rows (need these step for submission)

In [32]:
# Identify entries without valid text
test_dataset_invalid = test_dataset.filter(lambda example: not has_valid_text(example))

# Assign default prediction (assigning 0, room for exploration!)
default_prediction = 0  #

# Create predictions for entries without valid text
num_invalid = len(test_dataset_invalid)
test_predictions_invalid = [default_prediction] * num_invalid

# Merging empty rows with test predictions

In [33]:
# Initialize a list to hold all predictions
all_predictions = []

# Indexes for valid and invalid entries
valid_index = 0
invalid_index = 0

# Iterate over the original test dataset to reconstruct the predictions in order
for is_valid in valid_text_mask:
    if is_valid:
        # Append prediction from valid entries
        all_predictions.append(test_predictions_valid[valid_index])
        valid_index += 1
    else:
        # Append default prediction for invalid entries
        all_predictions.append(test_predictions_invalid[invalid_index])
        invalid_index += 1

# Sanity check, ensuring the number of predictions matches the number of test samples
assert len(all_predictions) == len(test_dataset), "Mismatch in number of predictions."


# sumbission file for keggle

In [34]:
# Extract 'id's from the test dataset
ids = test_dataset['id']

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'id': ids,
    'label': all_predictions
})

# Save the submission file
submission_df.to_csv('./my_submission_Roberta.csv', index=False)
